In [1]:
import pandas as pd
from data_loading import DataLoader
from preprocess import ConvertToMarkdown, SplitDocuments
from openai_utils import OpenAIEmbeddingClient, GenerateEmbeddings
from pinecone_utils import PineconeIndexClient, IngestEmbeddingsToPinecone
from oab_test_utils import OABTestFormatter, OABAnswersFormatter, OABTestFormatter41, OABAnswersFormatter43, OABAnswersFormatter39, OABQuestionAnswerFormatter
import os
from dotenv import load_dotenv
from pprint import pprint
import sys
sys.path.append(os.path.abspath(os.path.join('D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor')))
from utils.mongo_utils import ConnectAndInsertData

In [ ]:
load_dotenv()
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_ENVIRONMENT = os.getenv("PINECONE_ENVIRONMENT_REGION")
PINECONE_INDEX_NAME = os.getenv("PINECONE_INDEX_NAME")
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PDF_PATH = os.getenv("PDF_PATH")
PINECONE_NAMESPACE = os.getenv("PINECONE_NAMESPACE")
OPENAI_EMBEDDING_MODEL=os.getenv("OPENAI_EMBEDDING_MODEL")

In [2]:
numero_prova = '39'

In [3]:
gabarito_oab = ConvertToMarkdown(DataLoader(f'D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\{numero_prova}_gabarito.pdf'))
prova_oab = ConvertToMarkdown(DataLoader(f'D:\\Users\\Gabriel\\Desktop\\GIT\\pinecone-ingestor\\documentos_ingest\\provas\\{numero_prova}_prova.pdf'))
nome_prova = 'prova' + numero_prova

In [4]:
prova_formatada = OABTestFormatter(prova_oab, nome_prova)[0:79]
print(prova_formatada)

[{'pergunta': 'Mariana deseja ingressar no quadro da Sociedade de Advogados XYZ, na qualidade de associada, sem vínculo de emprego. Ao pesquisar a legislação que rege a parceria em questão, Mariana descobriu que constitui cláusula essencial do contrato de associação', 'a': 'a qualificação das partes, com referência expressa à inscrição no Conselho Seccional da OAB competente.', 'b': 'a identificação da parte que terá a responsabilidade exclusiva pelos riscos e pelas receitas decorrentes da prestação do serviço.', 'c': 'a forma de repartição da responsabilidade pelo fornecimento de condições materiais necessárias à execução dos serviços entre as partes, vedada a atribuição da totalidade das despesas exclusivamente a uma delas.', 'd': 'a estabilidade da parceria, materializada na ausência de prazo determinado para a duração do contrato.', 'prova': 'prova39'}, {'pergunta': 'Alice Santos, advogada, está sendo investigada criminalmente por ter, supostamente, cometido fraude contra o sistema

In [5]:
gabarito_formatado = OABAnswersFormatter39(gabarito_oab, nome_prova)
print(gabarito_formatado)

[{'pergunta': 1, 'resposta': 'a', 'prova': 'prova39'}, {'pergunta': 2, 'resposta': 'd', 'prova': 'prova39'}, {'pergunta': 3, 'resposta': 'd', 'prova': 'prova39'}, {'pergunta': 4, 'resposta': 'c', 'prova': 'prova39'}, {'pergunta': 5, 'resposta': 'a', 'prova': 'prova39'}, {'pergunta': 6, 'resposta': 'd', 'prova': 'prova39'}, {'pergunta': 7, 'resposta': 'b', 'prova': 'prova39'}, {'pergunta': 8, 'resposta': 'a', 'prova': 'prova39'}, {'pergunta': 9, 'resposta': 'd', 'prova': 'prova39'}, {'pergunta': 10, 'resposta': 'd', 'prova': 'prova39'}, {'pergunta': 11, 'resposta': 'c', 'prova': 'prova39'}, {'pergunta': 12, 'resposta': 'b', 'prova': 'prova39'}, {'pergunta': 13, 'resposta': 'a', 'prova': 'prova39'}, {'pergunta': 14, 'resposta': 'c', 'prova': 'prova39'}, {'pergunta': 15, 'resposta': 'a', 'prova': 'prova39'}, {'pergunta': 16, 'resposta': 'b', 'prova': 'prova39'}, {'pergunta': 17, 'resposta': 'b', 'prova': 'prova39'}, {'pergunta': 18, 'resposta': '*', 'prova': 'prova39'}, {'pergunta': 19, '

In [6]:
perguntas_respostas = OABQuestionAnswerFormatter(prova_formatada, gabarito_formatado)
print(perguntas_respostas)

[{'pergunta': 'Mariana deseja ingressar no quadro da Sociedade de Advogados XYZ, na qualidade de associada, sem vínculo de emprego. Ao pesquisar a legislação que rege a parceria em questão, Mariana descobriu que constitui cláusula essencial do contrato de associação', 'resposta': 'a qualificação das partes, com referência expressa à inscrição no Conselho Seccional da OAB competente.'}, {'pergunta': 'Alice Santos, advogada, está sendo investigada criminalmente por ter, supostamente, cometido fraude contra o sistema previdenciário, em conjunto com Robson Lima, seu cliente, e Leonardo Melo, seu ex-cliente. O órgão competente do Ministério Público consulta a Dra. Alice Santos sobre seu interesse em efetuar colaboração premiada. Com base na legislação aplicável, assinale a afirmativa que apresenta, corretamente, o que ela concluiu.', 'resposta': 'Caso efetue colaboração premiada contra Leonardo Melo, estará sujeita às penas do crime de violação do segredo profissional.'}, {'pergunta': 'Dura

In [ ]:
print("Conectando ao OpenAI...")
client = OpenAIEmbeddingClient(OPENAI_API_KEY)
index = PineconeIndexClient(PINECONE_API_KEY, PINECONE_ENVIRONMENT, PINECONE_INDEX_NAME)

In [ ]:
print("Gerando embeddings e enviando para Pinecone...")
count = 0
for pergunta_resposta in perguntas_respostas:
    pergunta = pergunta_resposta['pergunta']
    resposta = pergunta_resposta['resposta']
    text_to_insert = f'##pergunta: {pergunta} \n ##resposta: {resposta}'
    count+=1
    embedding = GenerateEmbeddings(client,OPENAI_EMBEDDING_MODEL, [text_to_insert])
    #chunk = 
    IngestEmbeddingsToPinecone([chunk], embedding, index, PINECONE_NAMESPACE)
print(count)


Gerando embeddings e enviando para Pinecone...
76
